In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
from tensorflow.keras.models import load_model

In [3]:
# Load trained model,scaler and onehot pickle file
model=load_model('model.h5')

#Load encoder and scaler
with open('label_encoded_gender.pkl','rb') as file:
    label_encoded_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [21]:
#Input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000

}

In [22]:
input_df=pd.DataFrame([input_data])
input_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [23]:
## Encode Gender (categorical feature)
input_df['Gender']=label_encoded_gender.transform(input_df['Gender'])


In [25]:
geo_encoder=onehot_encoder_geo.transform(input_df[['Geography']])
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [26]:
input_df=pd.concat([input_df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)
input_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [27]:
# Scaling hte input
input_scaled=scaler.transform(input_df)

In [28]:
#prediction
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


array([[0.03704422]], dtype=float32)

In [29]:
prediction_proba=prediction[0][0]

if prediction_proba > 0.5:
    print('The Customer is likely to churn')
else:
    print('The Customer is not likely to churn')

The Customer is not likely to churn
